<a href="https://colab.research.google.com/github/akankshakusf/Project-Fine-Tuned-LLMs-for-Product-Pricing/blob/master/Evaluate_My_Model_Predict_Product_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Predict Product Prices

### And now, to evaluate my  fine-tuned open source model

In [ ]:
# pip installs
!pip install -q datasets \
                requests \
                torch \
                peft \
                bitsandbytes \
                transformers \
                trl \
                accelerate \
                sentencepiece\
                tiktoken \
                matplotlib

In [ ]:
#import packages
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,set_seed
from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime
from peft import PeftModel
import matplotlib.pyplot as plt

In [ ]:
from ast import increment_lineno
#contants
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "akankshakusf"

#fine tuned model
RUN_NAME = "2025-05-02_20.00.01"
PROJECT_RUN_NAME =  f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = "5f1cd803dd15a87b779f73bc4061081e50e9e064" # or REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

#my data from hugging face
DATASET_NAME = f"{HF_USER}/pricer-data"


#hyperparameters for QLoRA
QUANT_4_BIT = True

%matplotlib inline

#used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}

In [ ]:
#log in to HuggingFace
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

In [ ]:
#load dataset
dataset = load_dataset(DATASET_NAME)
train= dataset['train']
test = dataset['test']#define quantization needed
if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# Load the Tokenizer and the Model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model with PEFT
if REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL, revision=REVISION)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)


print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")